# Aula - Árvores de Decisão para Regressão

- 1) Como adaptar o algoritmo das árvores de decisão para problemas de regressão?
- 2) Como avaliar meu modelo final de regressão? 

## 1) Árvores de Decisão para Regressão

Quando tínhamos o problema de classificação, nossa ideia era:<br>

> 1. Criar uma árvore de decisão em cima das minhas "features" e dos meus "labels". <br>
> 2. Para decidir os splits automaticamente, usamos critérios de impureza (Gini, entropia)  <br>
> 3. A classe prevista para um novo caso é dada pela folha que o novo ponto irá cair. <br>
> 4. A gente percorre a árvore, usando os dados desse novo ponto, e onde ela cair no fim (a folha final) decide a classe dela. <br>


<br><br><br><br><br><br><br><br><br><br>

A gente pode adaptar essa ideia para __problemas de regressão__. <br>

> 1. Criar uma árvore de decisão em cima das minhas "features" e dos meus "labels". <br>
> 2. Para decidir os splits automaticamente, usamos o quanto teríamos de erro se parássemos naquele nó. <br>
> 3. O valor previsto para um novo caso é o valor médio da folha que o novo ponto irá cair. <br>
> 4. A gente percorre a árvore, usando os dados desse novo ponto, e onde ela cair no fim (a folha final) decide o valor dele. <br>

Tivemos então que adaptar 2 pontos importantes:
1. Como decidir o split da árvore
2. Como decidir o valor final do nó

Vamos ver com um exemplo como faríamos isso.

In [ ]:
# importe as principais bibliotecas para ciência de dados
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

Vamos começar pelo ponto 3:<br>
A gente vai assumir que o __valor previsto para uma dada região é a média dos valores dos pontos de treino que estavam naquela região__.

In [ ]:
# Vamos pegar alguns pontos de exemplo
pontos = np.array([[1.0, 0.92],
                   [1.5, 0.51],
                   [2.0, -0.10],
                   [2.5, -0.02],
                   [3.0, 0.93],
                   [3.5, 2.18]])

In [ ]:
# Vamos desenhar eles, com um possível split inicial.
plt.title('Dados')
plt.scatter(pontos[:,0], pontos[:,1])
plt.xlabel('feature')
plt.ylabel('target')
plt.show()

In [ ]:
plt.title('Após 1º split')
plt.scatter(pontos[:,0], pontos[:,1])
plt.axvline(2.7, color='red')
plt.xlabel('feature')
plt.ylabel('target')
plt.show()

<font size=3> <b> Exercício 1: </b> </font>

1. Quais seriam os valores previstos do nó $ feature <= 2.7 $ (à esquerda da reta) se utilizássemos a média?
2. Quais seriam os valores previstos do nó $ feature > 2.7 $ (à esquerda da reta) se utilizássemos a média?
3. Esses valores previstos são bons? Como podemos fazer essa avaliação?
4. Qual era o melhor lugar para colocarmos aquela reta vermelha?
5. Onde você colocaria uma segunda quebra?
6. O que aconteceria com 5 quebras?
7. Como evitar esse problema?


Nossa função ficou da forma:

$$ y = \left\{
\begin{array}{ll}
    0.33 & , feature <= 2.7 \\
    1.55 & , feature > 2.7
\end{array} 
\right.  $$

In [ ]:
plt.title('Após 1º split')
plt.scatter(pontos[:,0], pontos[:,1])
plt.axvline(2.7, color='red')
plt.hlines(y=0.33, xmin=0.9, xmax=2.7, colors='aqua', linestyles='--', lw=2, label='0.33', alpha=0.7)
plt.hlines(y=1.55, xmin=2.7, xmax=3.6, colors='green', linestyles='--', lw=2, label='1.55', alpha=0.7)
plt.xlim([0.9, 3.6])
plt.xlabel('feature')
plt.ylabel('target')
plt.legend()
plt.show()

<br><br>

O que vamos fazer então é:
- Vamos escolher os splits que diminuíem a nossa métrica de erro
- Os valores que cada folha final da nossa árvore vai prever é a média de todos os pontos de treino que ficaram nela ao final.


Vamos considerar uma única feature gerada por uma função seno com ruído Gaussiano: se fizermos várias quebras na decision tree poderemos obter uma boa aproximação da curva real. 

<img src="images/decision-stump-1.png" style="width:400px" text="https://bradleyboehmke.github.io/HOML/DT.html">
<img src="images/decision-stump-2.png" style="width:400px" />

<br>
<img src="images/depth-3-decision-tree-1.png" style="width:400px" />
<img src="images/depth-3-decision-tree-2.png" style="width:400px" />
<br>
<img src="images/deep-overfit-tree-1.png" style="width:400px"/>
<img src="images/deep-overfit-tree-2.png" style="width:400px"/>

Isso nos leva à duas perguntas: 
* Como eu escolho qual é o melhor?
* Quando parar?

### Dataset

In [ ]:
from sklearn.datasets import load_boston
dados = load_boston()
X = pd.DataFrame(data=dados['data'], columns=dados['feature_names'])
y = pd.Series(data=dados['target'],name='price')

In [ ]:
print(dados['DESCR'])

In [ ]:
y.describe()

In [ ]:
# Sempre começar com a nossa famosa separação treino e validação
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, 
                                                  test_size=0.3)

Agora vamos treinar nossa árvore de decisão para regressão
<br>
[sklearn.tree.DecisionTreeRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html)

In [ ]:
# Importa algoritmo


# Instanciar a classe do algoritmo
model_tree = 

# Treina o modelo


In [ ]:
# Faz predições no conjunto de validação


In [ ]:
print(y_pred)

In [ ]:
# Como ficou a árvore?
from sklearn.tree import plot_tree
plt.figure(figsize=(20, 10))
plot_tree(model_tree, feature_names=X_train.columns, max_depth=3, fontsize=9, filled=True)
plt.show()


LSTAT aparece bastante nas quebras né? Porque será?

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Print das Métricas
print('Métricas para a Previsão:')
print('MAE:  ', np.round(mean_absolute_error(y_val, y_pred), decimals=3))
print('MSE:  ', np.round(mean_squared_error(y_val, y_pred), decimals=3))
print('RMSE: ', np.round(mean_squared_error(y_val, y_pred, squared=False), decimals=3))
print('R^2:  ', np.round(r2_score(y_val, y_pred), decimals=3))

In [ ]:
num_range = np.arange(0,50, 1)
sns.scatterplot(x = y_pred, y = y_val)
sns.lineplot(x=num_range, y=num_range, color='red', markers='-')
plt.xlabel('Predito')
plt.ylabel('Real')
plt.title('Comparação do valor predito com o real')

In [ ]:
# !pip install dtreeviz             # install dtreeviz for sklearn
# !pip install dtreeviz[xgboost]    # install XGBoost related dependency
# !pip install dtreeviz[pyspark]    # install pyspark related dependency
# !pip install dtreeviz[lightgbm]   # install LightGBM related dependency

In [ ]:
#! pip install dtreeviz

In [ ]:
from dtreeviz.trees import dtreeviz

model_tree = DecisionTreeRegressor(max_depth=3)
model_tree.fit(X_train, y_train)

viz = dtreeviz(model_tree,
               X_train,
               y_train,
               target_name='price',
               feature_names=list(X_train.columns))

viz   

#### Prós e Contras

Prós: <br>
        * Capaz de lidar com variáveis categóricas e contínuas <br>
        * Geram regras de fácil compreensão para o negócio e é muito intuitivo <br>
        * Não necessita de normalização dos dados nem da escala ([Normalização x Escala](https://kharshit.github.io/blog/2018/03/23/scaling-vs-normalization))  <br>
        * Não é obrigatório tratar dados faltantes (no scikit learn é haha)<br>
        * Por isso tem um EDA mais fácil <br>
        * Pode capturar relações não lineares <br>
        * Traz uma ideia da importância de cada feature<br>
        * Pouco sensível à outliers

Contras: <br>
        * Pode ser instável com pequenas mudanças nos dados - alta variância (pode ser corrigido com métodos de bagging e boosting) <br>
        * Datasets desbalanceados podem gerar um viés (bias) <br>
        * Por vezes demora mais para ser treinado que outros modelos <br>
        * Precisa de mais tempo de treino conforme aumenta o número de features <br>
        * Features contínuas geram aumento do tempo de treino <br>
        * Tende ao Overfiting

<br>
https://www.educba.com/decision-tree-advantages-and-disadvantages/


### Como superar esses problemas?
Random Forest

## Exercício

Vamos testar usar nossas árvores de decisão de regressão para o conjunto de dados "Ames Housing". Esse é um conjunto com diversas vendas de casas realizadas em Ames - Iowa.

O objetivo é prever o valor da venda de uma casa (SalePrice) com base nas features escolhidas. O conjunto de dados já foi previamente separado em treino e teste. Assim, só precisa tomar cuidado em separar o treino da validação.

Objetivos:
> Compare a qualidade preditiva da árvore de decisão para três conjuntos diferentes de variáveis (estes conjuntos podem ter interseções, ou seja, variáveis em comum). 
> Compare qual melhor modelo, entre uma árvore de decisão e outro modelo de regressão.

## Bibliografia e aprofundamento
- [Métricas](https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter)
- [Outras métricas](https://www.dataquest.io/blog/understanding-regression-error-metrics/) <br>
- [Scikit-Learn: Outras métricas de erro para regressão](https://scikit-learn.org/stable/modules/classes.html#regression-metrics) <br>
- [Biblioteca de visualização da árvore de decisão dtreeviz](https://github.com/parrt/dtreeviz) <br>
- [Visualização de árvores de decisão](https://mljar.com/blog/visualize-decision-tree/)
- [DT](https://www.cs.cornell.edu/courses/cs4780/2018fa/lectures/lecturenote17.html)
- [Are categorical variables getting lost in your random forests?](https://web.archive.org/web/20200924113639/https://roamanalytics.com/2016/10/28/are-categorical-variables-getting-lost-in-your-random-forests/)

<font size=5> <b> Tarefa: </b> </font>


Tente reproduzir o que fizemos aqui para outro modelo de regressão.

- As métricas melhoram ou pioram? <br>
- Olhando o RMSE, qual modelo você usaria? <br>
- Qual vai ser o RMSE no conjunto de testes? <br>